# Scattering for a circular metamaterial cavity

The codes below are associated to the article:

- C. Carvalho and Z. Moitier, _Asymptotics for metamaterial cavities and their effect on scattering_ [[arXiv](https://arxiv.org/abs/2010.07583), [HAL](https://hal.archives-ouvertes.fr/hal-02965993)]

We present computations related to Section 3 of the manuscript.

## Zoïs Moitier, Camille Carvalho (2021)
            
_Karlsruhe Institute of Technology, Germany_

_University of California Merced, USA_

In [1]:
from sys import argv

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FuncFormatter

from latex_plot import set_rcParams, set_size
from zNumeric import *

In [14]:
mrange = np.linspace(1,12,12, dtype=int)
print(mrange)

[ 1  2  3  4  5  6  7  8  9 10 11 12]


In [16]:
N = 512
H = 1.25
x = np.linspace(-H, H, num=N)
X, Y = np.meshgrid(x, x)

εcav = [-1.5, -1.3, -1.2, -1.1, -0.9, -0.8] 
δrange = np.linspace(0.01,0.1,10)
mrange = np.linspace(1,12,12, dtype=int)
from ipywidgets import interact 

from functools import lru_cache

@lru_cache
def calc(m, δ, ε):
    η = ε_to_η(ε)
    if ε < -1:
        data = np.load(f"../data/eps_{ε}.npz")
        pla = data["plasmon"]
        k0 = pla[m - 1, 1] 
    else:
        data = np.load(f"../data/eps_{-ε-2}.npz")
        pla = data["plasmon"]
        k0 = pla[m - 1, 1] 

    UU0 = np.abs(field_xy(η, k0, scattered_field_m, X, Y))
    UUp = np.abs(field_xy(η, k0+ δ, scattered_field_m, X, Y))
    UUm  = np.abs(field_xy(η, k0-δ, scattered_field_m, X, Y))
    return UU0, UUp, UUm, k0


@interact
def interact_plot_scattering(m:(1,12,1)=6, δ:(0.01, 0.1, 0.01)=0.01, ε = εcav):
    UU0, UUp, UUm, k0 = calc(m, δ, ε)
        
    plt.rcParams.update(set_rcParams(font_size=20, line_width=1))
    fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize = (30,10))
    Um_max = UUm.max()
    imm = ax1.pcolormesh(X, Y, UUm, shading="auto", vmin=0, vmax=Um_max)
    U0_max = UU0.max()
    im0 = ax2.pcolormesh(X, Y, UU0, shading="auto", vmin=0, vmax=U0_max)
    Up_max = UUp.max()
    imp = ax3.pcolormesh(X, Y, UUp, shading="auto", vmin=0, vmax=Up_max)
    fig.colorbar(imm, ax=ax1)
    fig.colorbar(im0, ax=ax2)
    fig.colorbar(imp, ax=ax3)

    ax1.set_aspect("equal", "box")
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.set_title(f"k-{δ}")

    ax2.set_aspect("equal", "box")
    ax2.set_xticks([])
    ax2.set_yticks([])
    ax2.set_title(f"k = {k0}")

    ax3.set_aspect("equal", "box")
    ax3.set_xticks([])
    ax3.set_yticks([])
    ax3.set_title(f"k+{δ}")

interactive(children=(IntSlider(value=6, description='m', max=12, min=1), FloatSlider(value=0.01, description=…

# Stability constant

In [3]:
εcav = [-1.5, -1.3, -1.2, -1.1, -0.9, -0.8, -0.75, -0.7] 
kMin = 0.01
kMax = 5
T = 2
                                    
@interact
def interact_plot_stability(ε1 = εcav, ε2 = εcav, m:(1,12,2)=4):                               
    data1 = np.load(f"../data/eps_{ε1}.npz")
    data2 = np.load(f"../data/eps_{ε2}.npz")

    vk1_ = sample_geo(kMin, kMax, 60, data1["plasmon"][:, 1], 3, 0.25)
    vk2_ = sample_geo(kMin, kMax, 60, data2["plasmon"][:, 1], 3, 0.25)
    
    if ε1 < -1:
        vk1 = vk1_
    else:
        vk1 = np.linspace(kMin, kMax, num=np.size(vk1_, 0))
    vn1 = calc_response(ε1, vk1, T)
    if ε2 < -1:
        vk2 = vk2_
    else:
        vk2 = np.linspace(kMin, kMax, num=np.size(vk2_, 0))
    vn2 = calc_response(ε2, vk2, T)
    
    plt.rcParams.update(set_rcParams(font_size=20, line_width=1.5))
    fig, (ax1, ax2) = plt.subplots(1,2, figsize = (10,5))
    ax1.semilogy(vk1, vn1, "C1", label=fr"$\varepsilon_{{\mathsf{{c}}}} = {ε1}$")
    ax2.semilogy(vk2, vn2, "C1", label=fr"$\varepsilon_{{\mathsf{{c}}}} = {ε2}$")
    
    Pla1 = data1["plasmon"]
    Im1 = Pla1[:, 0].astype(int)
    ind1 = np.where(Im1 == abs(m))[0]
    k1 = Pla1[ind1,1]
    ax1.plot([k1, k1], [0., 1], "C0--")
    ax1.text(k1, 0.03, fr"$k_{m}$", ha="center", va="top", color="C0")
    
    Pla2 = data2["plasmon"]
    Im2 = Pla2[:, 0].astype(int)
    ind2 = np.where(Im2 == abs(m))[0]
    k2 = Pla2[ind2, 1]
    ax2.plot([k2, k2], [0.0, 1], "C0--")
    ax2.text(k2, 0.03, fr"$k_{m}$", ha="center", va="top", color="C0")

    ax1.set_xlim(0, kMax)
    ax1.set_ylim(1e-1, 3e5)
    ax1.grid(True)
    ax1.legend(loc=2)
    ax2.set_xlim(0, kMax)
    ax2.set_ylim(1e-1, 3e5)
    ax2.grid(True)
    ax2.legend(loc=2)

interactive(children=(Dropdown(description='ε1', options=(-1.5, -1.3, -1.2, -1.1, -0.9, -0.8, -0.75, -0.7), va…